### Agentic Rag System

Query Reformulation: Instead of using the raw user query, the agent can craft optimized search terms that better match the target documents
Multi-Step Retrieval: The agent can perform multiple searches, using initial results to inform subsequent queries
Source Integration: Information can be combined from multiple sources like web search and local documentation
Result Validation: Retrieved content can be analyzed for relevance and accuracy before being included in responses

In [2]:
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever
from smolagents import CodeAgent, InferenceClientModel

class PartyPlanningRetrieverTool(Tool):
    name = "party_planning_retriever"
    description = "Uses semantic search to retrieve relevant party planning ideas for Alfred’s superhero-themed party at Wayne Manor."
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be a query related to party planning or superhero themes.",
        }
    }
    output_type = "string"

    def __init__(self, docs, **kwargs):
        super().__init__(**kwargs)
        self.retriever = BM25Retriever.from_documents(
            docs, k=5  # Retrieve the top 5 documents
        )

    def forward(self, query: str) -> str:
        assert isinstance(query, str), "Your search query must be a string"

        docs = self.retriever.invoke(
            query,
        )
        return "\nRetrieved ideas:\n" + "".join(
            [
                f"\n\n===== Idea {str(i)} =====\n" + doc.page_content
                for i, doc in enumerate(docs)
            ]
        )

# Simulate a knowledge base about party planning
party_ideas = [
    {"text": "A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.", "source": "Party Ideas 1"},
    {"text": "Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.", "source": "Entertainment Ideas"},
    {"text": "For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'", "source": "Catering Ideas"},
    {"text": "Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.", "source": "Decoration Ideas"},
    {"text": "Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.", "source": "Entertainment Ideas"}
]

source_docs = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in party_ideas
]

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(source_docs)

# Create the retriever tool
party_planning_retriever = PartyPlanningRetrieverTool(docs_processed)

# Initialize the agent
agent = CodeAgent(tools=[party_planning_retriever], model=InferenceClientModel())

# Example usage
response = agent.run(
    "Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options."
)

print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find ideas for a luxury superhero-themed party, including entertainment, catering, and decoration options.      │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  entertainment_ideas = party_planning_retriever(query="luxury superhero-themed party entertainment ideas")        
  print("Entertainment Ideas:", entertainment_ideas)                                                               
  catering_ideas = party_planning_retriever(query="luxury superhero-themed party catering ideas")                  
  print("Catering Ideas:", catering_ideas)                                                                         
  decoration_ideas = party_planning_retriever(query="luxury superhero-themed party decoration ideas")              
  print("Decoration Ideas:", decoration_ideas)                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Entertainment Ideas: 
Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.
Catering Ideas: 
Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.
Decoration Ideas: 
Retrieved ideas:


===== Idea 0 =====
A superhero-themed masquerade ball with luxury decor, including gold accents and velvet curtains.

===== Idea 1 =====
Interactive experiences with VR where guests can engage in superhero simulations or compete in themed games.

===== Idea 2 =====
Decorate with iconic superhero logos and projections of Gotham and other superhero cities around the venue.

===== Idea 3 =====
For catering, serve dishes named after superheroes, like 'The Hulk's Green Smoothie' and 'Iron Man's Power Steak.'

===== Idea 4 =====
Hire a professional DJ who can play themed music for superheroes like Batman and Wonder Woman.

Out: None

[Step 1: Duration 3.22 seconds| Input tokens: 2,065 | Output tokens: 139]

...

[Step 6: Duration 3.09 seconds| Input tokens: 23,289 | Output tokens: 996]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 7 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: 
Root=1-6846e5ce-331f0b58203032dc7b0c0a87;8f3ad240-48f0-43dc-9782-24c058bea6a4)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 7: Duration 0.16 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-6846e5ce-331f0b58203032dc7b0c0a87;8f3ad240-48f0-43dc-9782-24c058bea6a4)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.